# Iteration 4: Fix the sampling from lognormal distributions

## Plain English Summary

In this iteration we fix the sampling from lognormal distributions.  We create a function that converts the mean and standard deviation of the lognormal to the equivalent parameters of the underlying normal distribution.  This is because `numpy` requires the parameters to be in this format.



## Notebook imports

In [1]:
from IPython.display import Markdown, display
import math
from sim_tools.distributions import Lognormal

## LLM Prompt

In [2]:
display(Markdown("inputs/04_iteration.md"))

### Main command

modify all `acute_treatment `functions to call a function external to the class called `normal_moments_from_lognormal`. The function should accept the mean and standard deviation of the Lognormal distribution and return the mu, sigma of the underlying Normal distribution.  These returned values are the correct values to use when sampling from the Lognormal distributions. 

Show all of the model code: this includes all patient generator functions in the ASU class.

## Date and time of input to LLM

> 1530 2nd March 2024

## Observations on the LLM Output

* `normal_moments_from_lognormal` is defined correctly. 

## Sources reported by the LLM

* None given - Perplexity appears to have stopped providing them.

## LLM Output

In [3]:
import simpy
import numpy as np

# External function to convert Lognormal moments to Normal moments
def normal_moments_from_lognormal(log_mean, log_std):
    phi = np.sqrt(log_std ** 2 + log_mean ** 2)
    mu = np.log(log_mean ** 2 / np.sqrt(log_std ** 2 + log_mean ** 2))
    sigma = np.sqrt(np.log(phi ** 2 / log_mean ** 2))
    return mu, sigma

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            mu, sigma = normal_moments_from_lognormal(np.log(7.4), np.log(8.6))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        elif patient_type == "ESD":
            mu, sigma = normal_moments_from_lognormal(np.log(4.6), np.log(4.8))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        else:
            mu, sigma = normal_moments_from_lognormal(np.log(7.0), np.log(8.7))
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")

    def tia_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(np.log(1.8), np.log(5.0))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")

    def neuro_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(np.log(4.0), np.log(5.0))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")

    def other_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(np.log(3.8), np.log(5.2))
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            self.env.process(self.stroke_acute_treatment(post_asu_destination))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)



Stroke patient 1 arrived at 3.012769746464856 days and will go to Rehab
Stroke patient 1 finished treatment at 3.4125970518617246 days
Other patient 2 arrived at 3.4514041020933073 days
Stroke patient 3 arrived at 4.4234768283260895 days and will go to Other
Stroke patient 4 arrived at 5.2073210139452275 days and will go to Other
Stroke patient 5 arrived at 5.51629171920149 days and will go to Rehab
Stroke patient 6 arrived at 5.740707880155605 days and will go to Other
Stroke patient 7 arrived at 6.089967035226673 days and will go to Rehab
Stroke patient 7 finished treatment at 6.521141242699118 days
Other patient 7 finished treatment at 6.662097471996685 days
Stroke patient 7 finished treatment at 6.669637505389665 days
Stroke patient 7 finished treatment at 6.685597462135716 days
Stroke patient 8 arrived at 6.752391971065302 days and will go to ESD
Stroke patient 8 finished treatment at 7.178890705274981 days
Stroke patient 9 arrived at 7.41054887502221 days and will go to Other
Str

## Testing lognormal snippets

* Test of the lognormal sampling.  The code below is our own verified code. The source is given in the docstring.

In [4]:
# External function to convert Lognormal moments to Normal moments
def llm_normal_moments_from_lognormal(log_mean, log_std):
    phi = np.sqrt(log_std ** 2 + log_mean ** 2)
    mu = np.log(log_mean ** 2 / np.sqrt(log_std ** 2 + log_mean ** 2))
    sigma = np.sqrt(np.log(phi ** 2 / log_mean ** 2))
    return mu, sigma

In [5]:
def test_lognormal_moments(mean, std):
   
    # Lognormal class from sim-tools.
    expected_moments = Lognormal(mean, std)
    print(expected_moments.mu, expected_moments.sigma)
    
    ## llm code
    llm_mu, llm_sigma = llm_normal_moments_from_lognormal(mean, std)
    print(llm_mu, llm_sigma)
    
    return (llm_mu, llm_sigma) == (expected_moments.mu, expected_moments.sigma)

In [6]:
test_lognormal_moments(128.79, 267.51)

4.022977459259615 1.2924439729790718
4.022977459259615 1.2924439729790718


True

## Testing

* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods




### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [7]:
M = 10_000_000

In [8]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [9]:
test_1(M)

Stroke patient 1 arrived at 0.1849114855718149 days and will go to Other
Stroke patient 2 arrived at 0.2683351885698381 days and will go to Other
Stroke patient 2 finished treatment at 1.2470260995358071 days
Stroke patient 2 finished treatment at 1.2770842823135142 days
Stroke patient 3 arrived at 1.327058089518178 days and will go to Other
Stroke patient 4 arrived at 1.7078410085074425 days and will go to Other
Stroke patient 4 finished treatment at 1.9176957877088237 days
Stroke patient 4 finished treatment at 2.4720046725523255 days
Stroke patient 5 arrived at 6.035571117307027 days and will go to Rehab
Stroke patient 6 arrived at 7.103535190658814 days and will go to Other
Stroke patient 6 finished treatment at 7.309668108907623 days
Stroke patient 6 finished treatment at 7.676330441078931 days
Stroke patient 7 arrived at 8.077458913375215 days and will go to Rehab
Stroke patient 8 arrived at 8.791109249136262 days and will go to Rehab
Stroke patient 9 arrived at 9.52835193611665 

### Extreme value test 2

* The code is again modified so that all patient typs have a mean inter-arrival rate of $M$

**Result: PASS**

In [10]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [11]:
test_2(M)

0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [12]:
TEN_DAYS = 10

In [13]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    


In [14]:
test_3(TEN_DAYS)

Complex Neuro patient 1 arrived at 0.0921696323742468 days
Other patient 2 arrived at 0.25720254881717863 days
Other patient 3 arrived at 0.4368363789992922 days
Other patient 3 finished treatment at 0.6124675218829546 days
Stroke patient 4 arrived at 1.2181034719096233 days and will go to Other
Stroke patient 5 arrived at 1.4415759397429457 days and will go to Other
Other patient 6 arrived at 1.655948430394225 days
Other patient 6 finished treatment at 1.6628526603682885 days
Complex Neuro patient 6 finished treatment at 1.9128934800777024 days
Other patient 6 finished treatment at 2.0839014202591275 days
Stroke patient 7 arrived at 2.9049714242789326 days and will go to ESD
Stroke patient 8 arrived at 3.150690156650832 days and will go to Other
Stroke patient 8 finished treatment at 3.3495133999985294 days
Other patient 9 arrived at 3.3955163789433334 days
Other patient 9 finished treatment at 3.561869786794638 days
Stroke patient 9 finished treatment at 4.343814358584291 days
Stroke